In [ ]:
import cobra
import GEMS
import pandas as pd
import numpy as np
import os


import pickle
from etcpy import etc
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt

import requests

In [ ]:
data = GEMS.load_exp_data('../data/ExpGrowth.csv')

In [ ]:
path = '../'
params = pd.read_csv(os.path.join(path,'data/model_enzyme_params.csv'),index_col=0)

In [ ]:
def query_protein_length(proteins):
    base_url = "https://rest.uniprot.org/uniprotkb/"
    protein_lengths = {}
    
    for protein in proteins:
        url = base_url + protein + ".txt"
        response = requests.get(url)

        if response.status_code == 200:
            lines = response.text.split('\n')

            for line in lines:
                if line.startswith('SQ'):
                    parts = line.split()
                    protein_lengths[protein] = int(parts[2])

    return protein_lengths

In [ ]:
# Example usage
protein_list = params[params['Length'].isnull()].index.to_list()  # List of proteins for which you want to query the length
protein_lengths = query_protein_length(protein_list)

In [ ]:
# Example usage
protein_list = params[params['Length'].isnull()].index.to_list()  # List of proteins for which you want to query the length
protein_lengths = query_protein_length(protein_list)

#Print the protein lengths
for protein, length in protein_lengths.items():
    print(f"Protein: {protein} - Length: {length}")

In [ ]:
from concurrent.futures import ThreadPoolExecutor

In [ ]:
def query_protein_length(protein):
    base_url = "https://rest.uniprot.org/uniprotkb/"
    url = base_url + protein + ".txt"
    response = requests.get(url)

    if response.status_code == 200:
        lines = response.text.split('\n')

        for line in lines:
            if line.startswith('SQ'):
                parts = line.split()
                return int(parts[2])

    return None

In [ ]:
def fetch_protein_lengths(protein_list):
    protein_lengths = {}

    with ThreadPoolExecutor() as executor:
        # Submit the queries for protein lengths in parallel
        futures = [executor.submit(query_protein_length, protein) for protein in protein_list]

        # Retrieve the results as they become available
        for idx, future in enumerate(futures):
            protein = protein_list[idx]
            length = future.result()
            protein_lengths[protein] = length

    return protein_lengths

In [ ]:
protein_lengths2 = fetch_protein_lengths(protein_list)
protein_lengths2

In [ ]:
params['Length'] = params['Length'].fillna(protein_lengths)
params

In [ ]:
df = etc.calculate_thermal_params(params)

In [ ]:
mae = cobra.io.load_matlab_model('/home/aditi/Documents/Project/GECKO/userData/myEcoli/models/ecModel.mat')

In [ ]:
cobra.util.solvers

In [ ]:
mets = [met.id for met in mae.metabolites]
mets

In [ ]:
mae.metabolites.get_by_id('pi_c')

In [ ]:
mae.summary()

In [ ]:
reaction = cobra.Reaction('NGAM')
reaction.name = 'NGAM'
reaction.lower_bound = 3.2 # This is the default
reaction.upper_bound = 3.2  # This is the default

In [ ]:
reaction

In [ ]:
h_c = mae.metabolites.h_c
atp_c = mae.metabolites.atp_c
adp_c = mae.metabolites.adp_c
h2o_c = mae.metabolites.h2o_c
pi_c = mae.metabolites.pi_c

In [ ]:
reaction.add_metabolites({
    atp_c: -1.0,
    h2o_c: -1.0,
    adp_c: 1.0,
    h_c: 1.0,
    pi_c: 1.0
})

In [ ]:
mae.add_reactions([reaction])
print(len(mae.reactions))

In [ ]:
mae.reactions.NGAM

In [ ]:
dfae_batch = data[0].set_index('Ts').rename_axis(None)

In [ ]:
try: rae = etc.simulate_growth(mae,dfae_batch.index+273.15,df=df,sigma=0.5)
except: rae = np.zeros(dfae_batch.shape[0])

In [ ]:
rae = [0 if x is None else x for x in rae]
rae = [0 if x<1e-3 else x for x in rae]
print(rae)

In [ ]:
etc.simulate_growth(mae,dfae_batch.index+273.15,df=df,sigma=0.5)

In [ ]:
dfae_batch

In [ ]:
met = mae.metabolites.prot_pool

In [ ]:
T = 273.15+20.153299
Tadj=0

In [ ]:
for rxn in met.reactions:
    # this is to ignore reaction 'prot_pool_exchange': --> prot_pool
    if len(rxn.metabolites)<2: continue
    uniprot_id = rxn.id.split('_')[-1]
    cols = ['dHTH', 'dSTS','dCpu','Topt']
    [dHTH, dSTS,dCpu,topt]=df.loc[uniprot_id,cols]
    fNT = etc.get_fNT(T+Tadj,dHTH,dSTS,dCpu)
    if fNT < 1e-32: fNT = 1e-32
    new_coeff = rxn.metabolites[met]/fNT
    etc.change_rxn_coeff(rxn,met,new_coeff)

In [ ]:
map_kcatT(mae,T,df)

In [ ]:
def change_rxn_coeff(rxn,met,new_coeff):
    '''
    # This is based on the rxn.add_metabolites function. If there the metabolite is already in the reaction,
    # new and old coefficients will be added. For example, if the old coeff of metA is 1, use
    # rxn.add_metabolites({metA:2}), After adding, the coeff of metA is 1+2 = 3
    #
    '''

    diff_coeff = new_coeff-rxn.metabolites[met]
    if np.isnan(diff_coeff):
        diff_coeff = -9.999999999999999e+31
        
    try: rxn.add_metabolites({met:diff_coeff})
    except: rxn.add_metabolites({met:diff_coeff}, combine=False)

In [ ]:
def calculate_kcatT(T,dHTH,dSTS,dCpu,kcatTopt,dCpt,Topt):
    '''
    # Using Trainsition state theory to calculate kcat at temperature T.
    # dHTH, dSTS: entropy and enthalpy at comergence temperatures. Protein
    # unfolding process.
    # dCpu, heat capacity change unpon unfolding.
    # kcatTopt: kcat values at optimal temperature
    # Topt, optimal temperature of the enzyme, in K
    # T, temperature, in K
    #
    '''
    # Constants
    R = 8.314;
    TH = 373.5;
    TS = 385;
    T0 = 30+273.15;

    # Use the equation from solvedHT.m and re-organized
    dGuTopt = dHTH +dCpu*(Topt-TH) -Topt*dSTS-Topt*dCpu*np.log(Topt/TS);
    dHt = dHTH+dCpu*(Topt-TH)-dCpt*(Topt-T0)-R*Topt-(dHTH+dCpu*(Topt-TH))/(1+np.exp(-dGuTopt/(R*Topt)));

    # Calculate kcat at reference Temperautre
    kcat0 = kcatTopt/np.exp(np.log(Topt/T0)-(dHt+dCpt*(Topt-T0))/R/Topt+dHt/R/T0+dCpt*np.log(Topt/T0)/R);

    # Calculate kcat at given temperature
    kcatT = kcat0*np.exp(np.log(T/T0)-(dHt+dCpt*(T-T0))/R/T+dHt/R/T0+dCpt*np.log(T/T0)/R);

    return kcatT

In [ ]:
def map_kcatT(model,T,df):
    '''
    # Apply temperature effect on enzyme kcat.
    # based on trainsition state theory
    # model, cobra model
    # T, temperature, in K
    # df, a dataframe containing thermal parameters of enzymes: dHTH, dSTS, dCpu, Topt
    # Ensure that Topt is in K. Other parameters are in standard units.
    #
    # Gang Li, 2019-05-03
    #
    '''
    for met in model.metabolites:

        # look for those metabolites: prot_uniprotid
        if not met.id.startswith('prot_'): continue

        # ingore metabolite: prot_pool
        if met.id == 'prot_pool': continue
        uniprot_id = met.id.split('_')[1]

        # Change kcat value.
        # pmet_r_0001 + 1.8518518518518518e-07 prot_P00044 + 1.8518518518518518e-07 prot_P32891 -->
        # 2.0 s_0710 + s_1399
        #
        # 1.8518518518518518e-07 is correponding to 1/kcat
        # change the kcat to kcat(T)
        # In some casese, this coefficient could be 2/kcat or some other values. This doesn't matter.
        #
        # a protein could be involved in several reactions
        cols = ['dHTH', 'dSTS','dCpu','Topt','dCpt']
        [dHTH, dSTS,dCpu,Topt,dCpt]=df.loc[uniprot_id,cols]


        for rxn in met.reactions:
            if rxn.id.startswith('draw_prot'): continue

            # assume that Topt in the original model is measured at Topt
            kcatTopt = -1/rxn.metabolites[met]


            kcatT = calculate_kcatT(T,dHTH,dSTS,dCpu,kcatTopt,dCpt,Topt)
            if np.isnan(kcatT) or kcatT < 1e-32: kcatT = 1e-32
            new_coeff = -1/kcatT

            change_rxn_coeff(rxn,met,new_coeff)

In [ ]:
kcatT = 1e-32
new_coeff = -1/kcatT

In [ ]:
mae.reactions.ACM6PH